# Machine Learning


In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white' # Since I use a dark IDE

# To allow multiple outputs per cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix

from sklearn.model_selection  import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Data Preparation

## Reading in Data

Full data

In [2]:
wba_data = pd.read_csv("./Data/Normal/wba_data_CLEAN.csv")

Normal data

In [4]:
X_train = pd.read_csv("./Data/Normal/X_train.csv")
y_train = pd.read_csv("./Data/Normal/y_train.csv")
X_test = pd.read_csv("./Data/Normal/X_test.csv")
y_test = pd.read_csv("./Data/Normal/y_test.csv")

SMOTE data

In [6]:
X_train_smote = pd.read_csv("./Data/SMOTE/X_train_smote.csv")
y_train_smote = pd.read_csv("./Data/SMOTE/y_train_smote.csv")
X_test_smote = pd.read_csv("./Data/SMOTE/X_test_smote.csv")
y_test_smote = pd.read_csv("./Data/SMOTE/y_test_smote.csv")

Oversampling

In [8]:
X_train_over = pd.read_csv("./Data/Oversampling/X_train_over.csv")
y_train_over = pd.read_csv("./Data/Oversampling/y_train_over.csv")
X_test_over = pd.read_csv("./Data/Oversampling/X_test_over.csv")
y_test_over = pd.read_csv("./Data/Oversampling/y_test_over.csv")

## Deleting Unwanted Columns

In [10]:
all_dfs = [X_train, X_test]

,Gender,AgeY,ECV,TPP,SC,TympType,OAE1,OAE1.4,OAE2,OAE2.8,...,f(6168.8433),f(6349.6042),f(6535.6618),f(6727.1713),f(6924.2925),f(7127.1897),f(7336.0323),f(7550.9945),f(7772.2555),f(8000.0000)
0,1,8.583333,0.89,-151.0,2.53,1,0.0,5.0,9.2,11.7,...,0.3955,0.3583,0.2997,0.2709,0.2544,0.1956,0.1648,0.1158,0.1083,0.0542
1,1,9.833333,0.81,-63.0,1.32,0,8.1,20.8,14.7,6.9,...,0.4592,0.4006,0.3508,0.3381,0.2983,0.2789,0.2655,0.3118,0.3039,0.3022
2,0,10.083333,1.50,-28.0,0.56,0,10.4,15.4,15.6,9.9,...,0.2600,0.2409,0.2002,0.2064,0.2190,0.1961,0.2076,0.1994,0.2084,0.1099
3,1,8.666667,1.15,3.0,1.07,0,10.6,15.1,15.0,7.8,...,0.5128,0.4605,0.4126,0.4001,0.3634,0.3378,0.3084,0.3156,0.3042,0.2912
4,1,8.583333,1.03,-10.0,0.43,0,3.7,11.5,7.4,5.7,...,0.0000,0.0034,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
